In [1]:
from yelp.client import Client
import requests
import json
import pandas as pd
import numpy as np
from config import api_key
from config import google_key

In [2]:
data = []

headers = {'Authorization': 'Bearer %s' % api_key}

url='https://api.yelp.com/v3/businesses/search'

lat=44.9375
lon=-93.2010

for offset in range(0, 1000, 50):
    
    params = {
        'limit':50, 
        'latitude':lat,
        'longitude':lon,
        'radius':10000,
        'categories':'restaurants',
        'offset':offset
        }  
    
    response=requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        data += response.json()['businesses']
    elif response.status_code == 400:
        print('400 Bad Request')
        break
        
print(json.dumps(data, indent=4, sort_keys=True))

[
    {
        "alias": "caspers-and-runyons-nook-saint-paul",
        "categories": [
            {
                "alias": "divebars",
                "title": "Dive Bars"
            },
            {
                "alias": "burgers",
                "title": "Burgers"
            }
        ],
        "coordinates": {
            "latitude": 44.9265501,
            "longitude": -93.1567051
        },
        "display_phone": "(651) 698-4347",
        "distance": 3695.3041484773335,
        "id": "1qVOiuSGnldYG82YHq4S-w",
        "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/GgvZclP3xBy0iqDQTvWPvQ/o.jpg",
        "is_closed": false,
        "location": {
            "address1": "492 Hamline Ave S",
            "address2": "",
            "address3": "",
            "city": "Saint Paul",
            "country": "US",
            "display_address": [
                "492 Hamline Ave S",
                "Saint Paul, MN 55116"
            ],
            "state": "MN",
         

In [3]:
len(data)

1000

In [4]:
i=0
yelp_list=[]
for places in data:
    yelp_id=data[i]['id']
    name=data[i]['name']
    street=data[i]['location']['address1']
    city=data[i]['location']['city']
    zipcode=data[i]['location']['zip_code']
    address= f'{street}, {city} {zipcode}'
    rating=data[i]['rating']
    reviews=data[i]['review_count']
    latitude=data[i]['coordinates']['latitude']
    longitude=data[i]['coordinates']['longitude']
    if data[i]['is_closed']==False:
        business_dict={"YelpID":yelp_id,"Name":name,"Latitude":latitude,"Longitude":longitude,"Address":address, "Rating":rating,"Reviews":reviews}
        yelp_list.append(business_dict)
    i+=1
    
yelp_list

[{'YelpID': '1qVOiuSGnldYG82YHq4S-w',
  'Name': "Casper's and Runyon's Nook",
  'Latitude': 44.9265501,
  'Longitude': -93.1567051,
  'Address': '492 Hamline Ave S, Saint Paul 55116',
  'Rating': 4.5,
  'Reviews': 905},
 {'YelpID': 'ErvVFzAYVArZS7gcWgY0rg',
  'Name': "Matt's Bar",
  'Latitude': 44.9394683837891,
  'Longitude': -93.247428894043,
  'Address': '3500 Cedar Ave S, Minneapolis 55407',
  'Rating': 4.0,
  'Reviews': 1535},
 {'YelpID': 'G7iXpVSRuXzCbD2daJ_JYQ',
  'Name': 'Surly Brewing',
  'Latitude': 44.9732287,
  'Longitude': -93.2097663,
  'Address': '520 Malcolm Ave SE, Minneapolis 55414',
  'Rating': 4.0,
  'Reviews': 1110},
 {'YelpID': 'dMUv-dziQtYJ6Ca7q65XPw',
  'Name': 'Blue Door Pub St. Paul',
  'Latitude': 44.946732,
  'Longitude': -93.176774,
  'Address': '1811 Selby Ave, Saint Paul 55104',
  'Rating': 4.0,
  'Reviews': 785},
 {'YelpID': 'qgV2NGFS2ZL3JCwKTMkkPw',
  'Name': 'Birchwood Cafe',
  'Latitude': 44.95721,
  'Longitude': -93.2234,
  'Address': '3311 E 25th St

In [36]:
len(yelp_list)

1000

In [6]:
# yelp=pd.DataFrame(yelp_list)
# yelp.to_csv("yelp.csv")
# yelp

In [16]:
url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?'
google_data=[]

for places in yelp_list:
    
    params = {
        'key':google_key,
        'input':places['Name'],
        'inputtype':'textquery',
        'locationbias': 'point:' + str(places['Latitude']) + ", " + str(places['Longitude']),
        'radius': 10,
        'fields':'name,formatted_address,place_id,geometry,rating,user_ratings_total'
        }
    
    response = requests.get(url, params=params)

    if len(response.json()['candidates'])>0:
        google_data.append(response.json()['candidates'][0])
    else:
        google_data.append("")

google_data

[{'formatted_address': '492 Hamline Ave S, St Paul, MN 55116, United States',
  'geometry': {'location': {'lat': 44.92657029999999, 'lng': -93.1566689},
   'viewport': {'northeast': {'lat': 44.92792017989272,
     'lng': -93.15540827010727},
    'southwest': {'lat': 44.92522052010728, 'lng': -93.15810792989272}}},
  'name': 'The Nook',
  'place_id': 'ChIJeYtpNkgq9ocRm-fvoiveNzk',
  'rating': 4.6,
  'types': [],
  'user_ratings_total': 2491},
 {'formatted_address': '3500 Cedar Ave, Minneapolis, MN 55407, United States',
  'geometry': {'location': {'lat': 44.9394238, 'lng': -93.2476291},
   'viewport': {'northeast': {'lat': 44.94077367989272,
     'lng': -93.24614877010728},
    'southwest': {'lat': 44.93807402010728, 'lng': -93.24884842989273}}},
  'name': "Matt's Bar and Grill",
  'place_id': 'ChIJgzapGhIo9ocR1q6nNH0J_Xw',
  'rating': 4.6,
  'types': [],
  'user_ratings_total': 3999},
 {'formatted_address': '520 Malcolm Ave SE, Minneapolis, MN 55414, United States',
  'geometry': {'loc

In [25]:
len(google_data)

1000

i=0
google_list=[]

for places in google_data:
    if places != "":
        if "places_id" in places:
            google_id = places.get('place_id')
        if "name" in places:
            name = places.get('name')
        if "formatted_address" in places:
            address = places.get('formatted_address')
        if "rating" in places:
            rating  = places.get('rating')
        if "reviews" in places:
            reviews = places.get('user_ratings_total')
        if "geometry" in places:
            latitude = places['geometry']['location']['lat']
            longitude = places['geometry']['location']['lng']
        business_dict = {"Google Place ID":google_id,"Name":name,"Latitude":latitude,"Longitude":longitude,"Address":address, "Rating":rating,"Reviews":reviews}
    else:
        business_dict = {"Google Place ID":"","Name":"","Latitude":"","Longitude":"","Address":"", "Rating":"","Reviews":""}
    google_list.append(business_dict)
    i+=1
    
google_list

In [39]:
yelp_df = pd.DataFrame(yelp_list)
google_df=pd.DataFrame(google_list)

yelp_df.to_csv('yelp.csv')
google_df.to_csv('google.csv')

In [48]:
i - 0
compare_list=[]

for i in range(1000):

    compare = {"Yelp":yelp_list[i]['Name'],"Google":google_list[i]['Name']}
    compare_list.append(compare)
    i+=1

compare_df = pd.DataFrame(compare_list)
compare_df.to_csv('compare.csv')
compare_df

,Google,Yelp
0,The Nook,Casper's and Runyon's Nook
1,Matt's Bar and Grill,Matt's Bar
2,Surly Brewing Co.,Surly Brewing
3,Blue Door Pub St. Paul,Blue Door Pub St. Paul
4,Birchwood Cafe,Birchwood Cafe
5,The Naughty Greek,The Naughty Greek
6,Town Talk Diner & Gastropub,Town Talk Diner & Gastropub
7,Cecil's,Cecils
8,Hi-Lo Diner,Hi-Lo Diner
9,Butcher & The Boar,Butcher & The Boar
